In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

In [6]:
test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

In [19]:
print(len(train_dataset))
print(len(test_dataset))

60000
10000


In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [9]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [10]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    # softmaxは CrossEntropyLoss() で計算されるのでここではlogitのみ
    def forward(self, x):
        out = self.linear(x)
        return out

In [12]:
model = LogisticRegression(input_size, num_classes)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28 * 28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' % (
                epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data[0]))

Epoch: [1/5], Step: [100/600], Loss: 1.9912
Epoch: [1/5], Step: [200/600], Loss: 1.9045
Epoch: [1/5], Step: [300/600], Loss: 1.7971
Epoch: [1/5], Step: [400/600], Loss: 1.7851
Epoch: [1/5], Step: [500/600], Loss: 1.6578
Epoch: [1/5], Step: [600/600], Loss: 1.6309
Epoch: [2/5], Step: [100/600], Loss: 1.5966
Epoch: [2/5], Step: [200/600], Loss: 1.6407
Epoch: [2/5], Step: [300/600], Loss: 1.5471
Epoch: [2/5], Step: [400/600], Loss: 1.4943
Epoch: [2/5], Step: [500/600], Loss: 1.3414
Epoch: [2/5], Step: [600/600], Loss: 1.4125
Epoch: [3/5], Step: [100/600], Loss: 1.2640
Epoch: [3/5], Step: [200/600], Loss: 1.3282
Epoch: [3/5], Step: [300/600], Loss: 1.2186
Epoch: [3/5], Step: [400/600], Loss: 1.2231
Epoch: [3/5], Step: [500/600], Loss: 1.2083
Epoch: [3/5], Step: [600/600], Loss: 1.1223
Epoch: [4/5], Step: [100/600], Loss: 1.1858
Epoch: [4/5], Step: [200/600], Loss: 1.1639
Epoch: [4/5], Step: [300/600], Loss: 1.2159
Epoch: [4/5], Step: [400/600], Loss: 1.1308
Epoch: [4/5], Step: [500/600], L

In [25]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28 * 28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

In [26]:
correct

8264

In [27]:
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [28]:
torch.save(model.state_dict(), 'model.pkl')